In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..')) # or the path to your source code
sys.path.insert(0, module_path)
os.chdir('..')

In [7]:
from data.data import LoadData

dataset = LoadData("ZINC")

[I] Loading dataset ZINC...
train, test, val sizes : 10000 1000 1000
[I] Finished loading.
[I] Data load time: 5.5135s


In [47]:
import torch.nn as nn
from torch.utils.data import DataLoader
import torch
import numpy as np
from sklearn import svm
import dgl

In [11]:
num_atom_type = dataset.num_atom_type
hidden_dim = 106  # Updated to match the new configuration
embedding = nn.Embedding(num_atom_type, hidden_dim)

In [24]:
train_loader = DataLoader(dataset.train, batch_size=len(dataset.train), shuffle=True, collate_fn=dataset.collate)
test_loader = DataLoader(dataset.test, batch_size=len(dataset.test), shuffle=False, collate_fn=dataset.collate)
val_loader = DataLoader(dataset.val, batch_size=len(dataset.val), shuffle=False, collate_fn=dataset.collate)

In [35]:
def graph_info_hash(num_nodes, edge_index):
    edge_tuple = tuple(sorted(zip(edge_index[0].tolist(), edge_index[1].tolist())))
    return hash((num_nodes, edge_tuple))

def graph_hash(g):
    # Create hash from edges and number of nodes
    edges = g.edges()
    num_nodes = g.num_nodes()
    return graph_info_hash(num_nodes, edges)

In [37]:
# Load a precomputed eigenvector dataset using metis_import
from supp_data.molecules import metis_import
eig_dict = dict()
metis_import(eig_dict, graph_info_hash,
             {"train": "supp_data/molecules/zinc_train_rec_full.csv",
              "test": "supp_data/molecules/zinc_test_rec_full.csv",
              "val": "supp_data/molecules/zinc_val_rec_full.csv"},
             num_eigs=15,
             fixMissingPhi1=False,
             extraOrtho=True)

# eigenvectors are stored as nxe (technically, stored as (evals, evecs))

In [55]:
def get_eigenvectors(g, normalized_laplacian=False, eigval_norm="", num_eigs=64, eigmod=""):
    # NOTE: this uses L, not the normalized laplacian (maybe should be normalized?)
    adj = g.adjacency_matrix().to_dense()
    if normalized_laplacian:
        d_12 = torch.pow(adj.sum(dim=1), -0.5).view(1,-1)
        laplacian = torch.eye(adj.size(0)) - d_12 * adj * d_12.T
    else:
        laplacian = torch.diag(adj.sum(dim=1)) - adj
    torch_eigenvalues, torch_eigenvectors = torch.eig(laplacian, eigenvectors=True)

    # Sort on real component of eigenvalues, and remove complex part (should be 0)
    sort_indices = torch_eigenvalues[:, 0].argsort()
    torch_eigenvectors = torch_eigenvectors[:, sort_indices]
    torch_eigenvalues = torch_eigenvalues[:,0][sort_indices]

    if eigval_norm == "scale(-1,1)_all":
        torch_eigenvalues = torch_eigenvalues / torch_eigenvalues.max()
        torch_eigenvalues = 2 * torch_eigenvalues - 1
    elif eigval_norm == "scale(0,2)_all":
        torch_eigenvalues = torch_eigenvalues / torch_eigenvalues.max()
        torch_eigenvalues = 2 * torch_eigenvalues

    # Remove high-end spectrum
    torch_eigenvectors = torch_eigenvectors[:, :num_eigs].to(g.device)
    torch_eigenvalues = torch_eigenvalues[:num_eigs].to(g.device)
    
    if eigval_norm == "scale(0,2)_sub":
        # Scale the eigenvalues to [0, 2] range
        torch_eigenvalues = torch_eigenvalues / torch_eigenvalues.max()
        torch_eigenvalues = 2 * torch_eigenvalues
    
    # Replace the eigenvectors with a random orthonormal basis for the eigenspace, if requested
    if eigmod == "rand_basis":
        # Generate random vectors
        r = torch.randn((torch_eigenvectors.size(0), num_eigs), device=g.device)
        # Project into the eigenspace
        r = torch_eigenvectors @ (torch_eigenvectors.T @ r)
        # Orthonormalize
        for i in range(r.size(1)):
            for j in range(i):
                r[:, i] -= torch.dot(r[:, i], r[:, j]) * r[:, j]
            norm = torch.norm(r[:, i])
            if norm > 0:
                r[:, i] = r[:, i] / norm
        torch_eigenvectors = r
                
        # Generate eigenvalues as Rayleigh quotients
        quots = torch.einsum('ze,nz,ne->e', r, laplacian, r)
        if eigval_norm == "scale(-1,1)_all":
            quots = quots / quots.max()
            quots = 2 * quots - 1
        elif eigval_norm in ["scale(0,2)_all", "scale(0,2)_sub"]:
            quots = quots / quots.max()
            quots = 2 * quots
        torch_eigenvalues = quots[:num_eigs]
    
    # Pad with trailing zeros
    if torch_eigenvectors.size(1) < num_eigs:
        num_missing = num_eigs - torch_eigenvectors.size(1)
        vec_padding = torch.zeros((torch_eigenvectors.size(0), num_missing), device=g.device)
        val_padding = torch.zeros((num_missing,), device=g.device)
        torch_eigenvectors = torch.cat((torch_eigenvectors, vec_padding), dim=1)
        torch_eigenvalues = torch.cat((torch_eigenvalues, val_padding), dim=0)

    # Save the computation results and return
    return (torch_eigenvalues.detach(), torch_eigenvectors.detach())

In [56]:
regr = svm.SVR()
for iter, (batch_graphs, batch_targets) in enumerate(train_loader):
    batch_x = batch_graphs.ndata['feat']
    batch_e = batch_graphs.edata['feat']
    
    batch_x = embedding(batch_x)
    
    # split the batch by graph (each entry is nxf)
    batch_x = batch_x.split(tuple(batch_graphs.batch_num_nodes()))
    graphs = dgl.unbatch(batch_graphs)
    
    # transform to eigencomponents (exf)
    #print(eig_dict[graph_hash(g)][1].shape)
    #evecs = [eig_dict[graph_hash(g)][1] for g in graphs]
    evecs = [get_eigenvectors(g, num_eigs=15)[1] for g in graphs]
    batch_x = [evec.T @ x for evec, x in zip(evecs, batch_x)]
    
    # regress on the graphs, flattening each set of features
    batch_x = [x.flatten() for x in batch_x]
    batch_x = torch.stack(batch_x, dim=0)
    regr.fit(batch_x.detach().numpy(), batch_targets.detach().numpy())
    

/home/bsw38/.conda/envs/benchmark_gnn_2/lib/python3.7/site-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/bsw38/.conda/envs/benchmark_gnn_2/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


In [57]:
for iter, (batch_graphs, batch_targets) in enumerate(test_loader):
    batch_x = batch_graphs.ndata['feat']
    batch_e = batch_graphs.edata['feat']
    
    batch_x = embedding(batch_x)
    
    # split the batch by graph (each entry is nxf)
    batch_x = batch_x.split(tuple(batch_graphs.batch_num_nodes()))
    graphs = dgl.unbatch(batch_graphs)
    
    # transform to eigencomponents (exf)
    #print(eig_dict[graph_hash(g)][1].shape)
    #evecs = [eig_dict[graph_hash(g)][1] for g in graphs]
    evecs = [get_eigenvectors(g, num_eigs=15)[1] for g in graphs]
    batch_x = [evec.T @ x for evec, x in zip(evecs, batch_x)]
    
    # regress on the graphs, flattening each set of features
    batch_x = [x.flatten() for x in batch_x]
    batch_x = torch.stack(batch_x, dim=0)
    
    predictions = regr.predict(batch_x.detach().numpy())
    targets = batch_targets.detach().numpy()

In [58]:
errs = predictions - targets

In [59]:
np.mean(np.abs(errs))

1.8087695249993212